# Homework 4: Problem 15

A simple electronic circuit pictured in Fig. 2 contains one resistor and one capacitor.
This circuit acts as a low-pass filter: you send a signal in on the left, and it comes out filtered on the right. Using Ohm’s law and the laws governing capacitors, and assuming that the output load has very high impedance (so that a negligible amount of current flows through it), we can write down the equations governing this circuit as follows. Let I be the current that flows through R and into the capacitor, and let Q be the charge on the capacitor. Then

$$ IR = V_{in} − V_{out}$$
$$ Q = CV_{out} $$
$$ I = \frac{dQ}{dt} $$

Substituting the second equation into the third, then substituting that result into the first equation, we find that the voltage dynamics can be described as
$$ \frac{dV_{out}}{dt} = \frac{1}{RC}(V_{in}-V_{out}). $$

A) Write a program to solve this equation for Vout (t) using the fourth-order Runge-Kutta method when the input signal is a square wave with frequency 1 and amplitude 1, i.e.
$$ V_{in}(t)=\left\{
    \begin{array}{rl}
1&\mbox{if }\lfloor 2t\rfloor \mbox{ is even}\\
-1&\mbox{if }\lfloor 2t \rfloor \mbox{ is odd}\\
\end{array}
\right.
$$
Use the program to make plots of the output of the filter circuit from $t = 0$ to $t = 10$ when R = 0.01, 0.1, and 1, with initial condition $V_{out} = 0.$ You will have to make a decision about what value of ∆t to use in your calculation. Small values give more accurate results, but the program will take longer to run. Try a variety of different values and choose one for your final calculations that seems sensible to you.

In [ ]:
from nm4p import rk4
import numpy as np
def square_wave(t):
    """Returns the specified square wave when give the time in seconds
    Params:
    t - the current time in seconds
    Returns:
        The square wave defined  evaluated at time t"""
    sqrwave = (-1)**(np.floor(2*t)% 2)
    return sqrwave

def v_out_derivs(x, t, r, c, v_in):
    """Generates the derivative for use in the rk4 routine at using the given parameters, such as the time to evaluate the derivative
    Params:
    x - the current value of V_out
    t- the current time t
    R - the resistance of the resistor in the circuit
    C- the Capacitance of the capacitor in the circuit
    V_in - the input voltage function which to be evaluated at the current time t """
    deriv = 1/(r*c)*(v_in(t)-x)
    return deriv
def lowpass_circuit(duration,v_in,v_out_init,r,c):
    """A circuit that simulates and plots a low pass filter using rk4 and hte differential equaiton provided above

    Params:
    duration - how many simulated seconds the simulation should be evaluated for
    v_in - a function v_in(t) that specifies the input voltage the simulator should use
    v_out_init - the initial value for the output voltage of hte circuit-- v_out at t=0
    r - the resistance of the resistor in the simulated circuit
    c - the capacitance of the capacitor in the simulated circuit
    Returns:
        returns no output but plots v_out from time 0 to the specified number of simulated seconds
    """


B) Based on the graphs produced by your program, describe what you see and explain what the circuit is doing.